# Theory and Concepts: Understanding LLMs Before You Trust Them

This notebook introduces the conceptual foundations of Large Language Models (LLMs) in research contexts. Before working with APIs, it's crucial to understand what LLMs are, what they can and cannot do, and how to use them responsibly.

We will cover:
- What LLMs are and how they work
- Strengths and weaknesses in research use
- Basics of **prompt engineering**
- Why **hallucination** occurs and how to detect it
- Benefits and risks across use cases
- Concept of **Agentic AI**
- Running LLMs via **API vs locally**

## 1. What Is a Large Language Model (LLM)?

An LLM is a **probabilistic text generator**. It predicts the next word in a sequence based on all the previous words.

Formally, it learns to estimate:

$$p(x_{t+1} \mid x_{1:t}) = \text{softmax}\left(\frac{z_i}{T}\right)$$

where $z_i$ are logits (raw scores), and $T$ is temperature controlling randomness.

**Key ideas:**
- The model is not reasoning; it's *pattern matching*.
- It is trained on billions of text samples.
- It does not know whether statements are *true*.

💡 **Analogy:** LLMs complete text the way autocomplete finishes a sentence — just on a massive scale.

## 2. What LLMs Are Good For — and What They Are Not

| ✅ Good At | ⚠️ Not Good At |
|-------------|---------------|
| Summarising academic text | Producing verified facts |
| Paraphrasing and rewriting | Mathematical proofs or derivations |
| Explaining code or methods | Statistical inference without data |
| Generating boilerplate writing | Handling private or sensitive data |
| Brainstorming research ideas | Acting as a source of truth |

Think of an LLM as an *assistant*, not a *co-author*.

## 3. Prompt Engineering Essentials

Prompt engineering means **crafting the input** to shape the model's response.

### Example
```
❌ Bad: Explain machine learning.
✅ Better: In three sentences, explain machine learning to a biology PhD student unfamiliar with computer science.
```

### Best Practices
- Be **specific** about role, audience, and format.
- Use **system messages** to set tone or constraints.
- Break complex queries into smaller sub-prompts.
- Ask for **structured output** (e.g., JSON, tables).

Let’s illustrate how different prompt structures might be interpreted.

In [1]:
# Demonstration: Prompt variety examples (no API calls)
# This cell just prints examples and explanations of what makes prompts effective.

example_prompts = [
    ("Explain AI.", "Too vague — model may produce generic output."),
    ("Explain Artificial Intelligence in two sentences for an interdisciplinary audience.", "Better — specifies length and audience."),
    ("As a data scientist, summarise Artificial Intelligence focusing on statistical learning methods.", "Excellent — adds role and context, leading to relevant focus.")
]

for text, comment in example_prompts:
    print(f"Prompt: {text}\nComment: {comment}\n{'-'*70}")

Prompt: Explain AI.
Comment: Too vague — model may produce generic output.
----------------------------------------------------------------------
Prompt: Explain Artificial Intelligence in two sentences for an interdisciplinary audience.
Comment: Better — specifies length and audience.
----------------------------------------------------------------------
Prompt: As a data scientist, summarise Artificial Intelligence focusing on statistical learning methods.
Comment: Excellent — adds role and context, leading to relevant focus.
----------------------------------------------------------------------


### Reflection
- How would you phrase prompts for your research area?
- What happens if your question is ambiguous or underspecified?
- How might prompt reproducibility affect research transparency?

## 4. Why Hallucination Happens

**Hallucination** is when the model produces false but plausible information.

**Causes:**
1. Models optimise for *fluency*, not *truth*.
2. They lack external knowledge verification.
3. They use patterns, not evidence.

**Mitigations:**
- Use grounded prompts (e.g., with text context or retrieval).
- Ask for *sources* and check them.
- Rephrase prompts to encourage uncertainty awareness (e.g., “If unsure, say so.”)

In [2]:
# Simulating hallucination detection with a fabricated response
# This code checks whether a response contains uncertainty words.

response = "Dr. Jane Quantum won the Nobel Prize in Quantum Psychology in 2024."

uncertainty_markers = ["might", "possibly", "may", "perhaps", "uncertain"]
uncertain = any(word in response.lower() for word in uncertainty_markers)

if uncertain:
    print("✅ The statement expresses uncertainty.")
else:
    print("⚠️ This response shows *overconfidence* — likely hallucination.")

⚠️ This response shows *overconfidence* — likely hallucination.


## 5. Benefits and Risks in Research

| Application | Benefit | Risk / Limitation |
|--------------|----------|------------------|
| Literature summarisation | Saves time, finds patterns | Hallucinated facts or missing nuance |
| Coding help | Faster prototyping | Wrong syntax or unsafe imports |
| Academic writing | Better grammar, flow | Style drift, plagiarism concerns |
| Brainstorming ideas | Expands creativity | May output unverified claims |
| Data cleaning | Quick suggestions | May fabricate column names |

🧭 Always cross-check LLM-generated outputs before citing or integrating into research.

## 6. Agentic AI (Concept Only)

**Agentic AI** refers to models that can take *initiative* — plan actions, call tools, and iteratively refine results.

**Examples:** AutoGPT, LangChain Agents, ChatGPT with code or browsing.

They combine:
- **Planning** (deciding what to do next)
- **Memory** (recalling previous steps)
- **Tool use** (e.g., running Python or querying databases)

### Why it matters
- Moves from passive Q&A to *autonomous workflows*.
- Raises questions of **accountability** and **control**.

🧩 The RAG (Retrieval-Augmented Generation) notebook later in this series is a *small step* toward agentic systems.

## 7. Running LLMs via API vs Locally

### Comparison

| Approach | Pros | Cons |
|-----------|------|------|
| **API (Groq, OpenAI)** | No setup, scalable, always updated | Requires internet, API cost, data privacy concerns |
| **Local (Hugging Face, vLLM)** | Full control, offline | Needs high VRAM, complex setup |

Below we estimate how much GPU memory is needed to run different models locally.

In [3]:
# Estimate GPU VRAM requirement for hosting models locally
# Rule of thumb: 1 billion parameters ≈ 2 GB VRAM (16-bit precision)

def estimate_vram(params_billion, precision_bits=16):
    """Estimate GPU memory needed for model parameters."""
    bytes_per_param = precision_bits / 8
    total_gb = (params_billion * 1e9 * bytes_per_param) / (1e9 * 1.024)
    return round(total_gb, 1)

models = {"Llama-3 8B": 8, "Llama-3 70B": 70, "Mistral 7B": 7}

print("Approximate VRAM needed (16-bit precision):\n")
for model, size in models.items():
    print(f"{model:15s}: {estimate_vram(size)} GB")

Approximate VRAM needed (16-bit precision):

Llama-3 8B     : 15.6 GB
Llama-3 70B    : 136.7 GB
Mistral 7B     : 13.7 GB


➡️ A 70B model would require well over **140 GB of GPU VRAM**, so APIs are currently the most practical solution for most researchers.

## 8. Pros and Cons Summary of using API

| Aspect | Pros | Cons |
|---------|------|------|
| Ease of use | Minimal setup | Reliance on vendor uptime |
| Cost | Free/cheap for small workloads | Expensive at scale |
| Reproducibility | Controlled APIs | Model updates may change outputs |
| Ethics | Accessible to all | Privacy and bias concerns |

Balance practicality with reproducibility — document all model versions and API calls when publishing results.

## 9. Reflection

**Questions for you:**
1. Which tasks in your workflow could LLMs assist with?
2. What risks might arise from automation in your field?
3. How can you document LLM involvement transparently in your papers?

Write your reflections below as markdown cells.

## ✅ Summary
- LLMs predict words, not truths — verification is essential.
- Prompt engineering is key to consistent behaviour.
- Hallucinations are unavoidable but detectable.
- Agentic AI is emerging; RAG is its foundation.
- APIs simplify use; local models give control but need hardware.

